<a href="https://colab.research.google.com/github/Locke0/Text-Summarizer/blob/main/Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#imports
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
# loading data
import requests


#Helpful Links

https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/

https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70

# Data Processing


## Loading data

In [23]:
# text: https://www.gutenberg.org/cache/epub/21077/pg21077.txt
# tutorial: https://www.youtube.com/watch?v=mKP7ChzJBqg


# loading books

result = requests.get('https://www.gutenberg.org/cache/epub/21077/pg21077.txt')
text = result.text



##Generate clean sentences

In [22]:
def read_article(text):
  article = text.split(". ")
  sentences = []
  for sentence in article:
    print(sentence)
    #deleting any character that IS NOT a-z OR A-Z
    sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()    
  return sentences


# def read_book(file_name):
  #   file = open(file_name, "r")
  #   filedata = file.readlines()
  #   article = filedata[0].split(". ")
  #   sentences = []
  #  for sentence in article:
  #    print(sentence)
  #    sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
  #    sentences.pop()  
    
    # return sentences

##Similarity matrix